In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
from datetime import datetime

In [ ]:
start = datetime.now()

In [ ]:
df = pd.DataFrame({'Address': list(), 'Size':list(), 'Rooms':list(), 'Floor':list(), 
                   'Price (eur)':list(), 'Price_by_surface (eur/m2)':list(), 'Type':list()})
l = []

In [ ]:
url = 'https://www.halooglasi.com/nekretnine/prodaja-stanova/beograd?page='

# headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'} 

# site currently does not reqest usage of headers

for page in range (1, 50): #pages to scrap, starting from page 1 to n

    r = get(f'{url}{page}') # headers= headers) 
    c = r.content
    soup= BeautifulSoup(c, 'html.parser')

    cases =['Premium', 'Standard', 'Top']

# There are 3 different add cases, depanding if apartment is classified as a Premium, Top or Standared.
# In html, only the division class product changes, while all other things are same

    for case in cases:
        find_all = soup.find_all('div', {'class':f'product-item product-list-item {case} real-estates my-product-placeholder'}) 

        for entry in find_all:
            d = {}
            try:
                d['Address'] = entry.find('ul', {'class': 'subtitle-places'}).text.replace('\xa0', ' ')
            except:
                d['Adress']= None
            
            
# Serbian numbering notation is a bit different than english, thus python interpretates values 
# larger then 1000 as integer (1.0). That is why we need to check their type and scale them accordinlgy.
# which usually is the case. If the integer check is ommited, values smaller then 1000 would be misinterpretated
# for example 858 would be interpretated as 858000
    
            
            try:
                d['Price (eur)'] = int(entry.find('div', {'class':'central-feature'}).text.replace('\xa0', ' ').replace(' €', ''))
            except:
                try:
                    d['Price (eur)'] = 1000*float(entry.find('div', {'class':'central-feature'}).text.replace('\xa0', ' ').replace(' €', ''))
                except:
                    d['Price (eur)']= None
                
            try:
                d['Price_by_surface (eur/m2)'] = int(entry.find('div', {'class':'price-by-surface'}).text.replace('\xa0', ' ').replace('€/m2', ''))
            except:
                try:
                    d['Price_by_surface (eur/m2)'] = 1000*float(entry.find('div', {'class':'price-by-surface'}).text.replace('\xa0', ' ').replace('€/m2', ''))
                
                except:
                    d['Price_by_surface (eur/m2)']= None
            
            try:
                d['Size'] = entry.find_all('div', {'class':'value-wrapper'})[0].text.replace('\xa0', ' ').replace('Kvadratura', '')
            except:
                d['Size']= None
            
            try:
                d['Rooms'] = entry.find_all('div', {'class':'value-wrapper'})[1].text.replace('\xa0', ' ').replace(' Broj soba', '')
            except:
                d['Rooms']= None
            
            try:
                d['Floor'] = entry.find_all('div', {'class':'value-wrapper'})[2].text.replace('\xa0', ' ').replace(' Spratnost', '')
            except:
                d['Floor']= None
            d['Type'] = case
            
            l.append(d) #appending to the list, and not directly to the data frame, becase it should be faster this way

In [ ]:
df = df.append(l, ignore_index=True)

In [ ]:
df.to_csv('StanoviBGD.csv')

In [ ]:
print(datetime.now() - start)